In [21]:
import torch
import torch.nn as nn
import os
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
from torch.optim import Adam
from torch.optim import LBFGS
import matplotlib.pyplot as plt
import pandas as pd

In [22]:
def activation(name):
    if name in ['tanh', 'Tanh']:
        return nn.Tanh()
    elif name in ['relu', 'ReLU']:
        return nn.ReLU(inplace=True)
    elif name in ['lrelu', 'LReLU']:
        return nn.LeakyReLU(inplace=True)
    elif name in ['sigmoid', 'Sigmoid']:
        return nn.Sigmoid()
    elif name in ['softplus', 'Softplus']:
        return nn.Softplus(beta=4)
    elif name in ['celu', 'CeLU']:
        return nn.CELU()
    elif name in ['elu']:
        return nn.ELU()
    elif name in ['mish']:
        return nn.Mish()
    else:
        raise ValueError('Unknown activation function')

################################################################
#  1d fourier layer
################################################################
class SpectralConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, modes1):
        super(SpectralConv1d, self).__init__()

        """
        1D Fourier layer. It does FFT, linear transform, and Inverse FFT.    
        """

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.modes1 = modes1

        self.scale = (1 / (in_channels * out_channels))
        self.weights1 = nn.Parameter(self.scale * torch.rand(in_channels, out_channels, self.modes1, dtype=torch.cfloat))

    # Complex multiplication
    def compl_mul1d(self, input, weights):
        # (batch, in_channel, x ), (in_channel, out_channel, x) -> (batch, out_channel, x)
        return torch.einsum("bix,iox->box", input, weights)

    def forward(self, x):
        # x.shape == [batch_size, in_channels, number of grid points]
        d = x.shape[2]
        # Compute Fourier coefficients
        x = torch.fft.rfft(x,dim=2)
        # Multiply Fourier modes
        x = self.compl_mul1d(x, self.weights1)
        # Return to physical space
        x = torch.fft.irfft(x, n=d)
        return x


####################################################################

class FNO1d(nn.Module):
    def __init__(self, modes, width):
        super(FNO1d, self).__init__()

        """
        The overall network. It contains 4 layers of the Fourier layer.
        1. Lift the input to the desire channel dimension by self.fc0 .
        2. 4 layers of the integral operators u' = (W + K)(u).
            W defined by self.w; K defined by self.conv .
        3. Project from the channel space to the output space by self.fc1 and self.fc2 .

        input: the solution of the initial condition and location (a(x), x)
        input shape: (batchsize, x=s, c=2)
        output: the solution of a later timestep
        output shape: (batchsize, x=s, c=1)
        """

        self.modes1 = modes
        self.width = width
        self.padding = 1  # pad the domain if input is non-periodic
        self.linear_p = nn.Linear(3, self.width)  # input channel is 2: (u0(x), x)

        self.spect1 = SpectralConv1d(self.width, self.width, self.modes1)
        self.spect2 = SpectralConv1d(self.width, self.width, self.modes1)
        self.spect3 = SpectralConv1d(self.width, self.width, self.modes1)
        self.lin0 = nn.Conv1d(self.width, self.width, 1)
        self.lin1 = nn.Conv1d(self.width, self.width, 1)
        self.lin2 = nn.Conv1d(self.width, self.width, 1)

        self.linear_q = nn.Linear(self.width, 32)
        self.output_layer = nn.Linear(32, 1)

        self.activation = torch.nn.Tanh()

    def fourier_layer(self, x, spectral_layer, conv_layer):
       # return self.activation(spectral_layer(x))
        return self.activation(spectral_layer(x) + conv_layer(x))
        
                       
    def linear_layer(self, x, linear_transformation):
        return self.activation(linear_transformation(x))

    def forward(self, x):
        # grid = self.get_grid(x.shape, x.device)
        # x = torch.cat((x, grid), dim=-1)
        x = self.linear_p(x)
        x = x.permute(0, 2, 1)
        # x = F.pad(x, [0, self.padding])  # pad the domain if input is non-periodic

        x = self.fourier_layer(x, self.spect1, self.lin0)
        x = self.fourier_layer(x, self.spect2, self.lin1)
        x = self.fourier_layer(x, self.spect3, self.lin2)

        # x = x[..., :-self.padding]  # pad the domain if input is non-periodic
        x = x.permute(0, 2, 1)

        x = self.linear_layer(x, self.linear_q)
        x = self.output_layer(x)
        return x


In [23]:
class DatasetTask3(TensorDataset):
    # https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
    # devide data into 5 input/outpus pairs, for cycle prediction
    def __init__(self, tensor_data, window_len):
        self.tensor_data = tensor_data
        self.window_len = window_len    # size of input/output sequences

    def __len__(self):
        # gives number of input and output pairs. In our case it is 5.
        return int(len(self.tensor_data)/self.window_len) - 1
        # return (len(self.data) // self.seq_length) - 1

    def __getitem__(self, idx):
        # idx is index of individual input-output pair
        lower_end = idx*self.window_len
        upper_end = (idx + 1)*self.window_len
        inputs = self.tensor_data[lower_end:upper_end]
        outputs = self.tensor_data[upper_end:upper_end+self.window_len,1:]
        
        return inputs, outputs

In [36]:
def normalize(x):
    """ mins and maxs for backconversion in lists"""
    t= x[:,0]; Tf= x[:,1] ; Ts = x[:,2]
    
    max_t , max_Tf, max_Ts = t.max(), Tf.max(), Ts.max()
    min_t , min_Tf, min_Ts = t.min(), Tf.min(), Ts.min()
    
    normalized_data = torch.zeros_like(x)
    normalized_data[:, 0] = (t - min_t)/(max_t - min_t)
    normalized_data[:, 1] = (Tf - min_Tf)/(max_Tf - min_Tf)
    normalized_data[:, 2] = (Ts - min_Ts)/(max_Ts - min_Ts)

    return normalized_data, [min_t , min_Tf, min_Ts], [max_t , max_Tf, max_Ts]

def denormalize(x, min, max):
    data = torch.zeros_like(x)
    for i in range(len(min)):
        data[:,i]= x[:,i]*(max[i]- min[i])+ min[i]
    return data

def regularization(model, order):
    reg_loss = 0
    for name, param in model.named_parameters():
        if 'weight' in name or 'bias' in name:
            reg_loss = reg_loss + torch.norm(param, order)
    return reg_loss

def fit_(fno_f, fno_s,epochs, optimizer, training_set, regularization_p):  
        history = list()
        l = torch.nn.MSELoss()
    
        for epoch in range(epochs):
            train_mse = 0.0
            for step, (input_batch, output_batch) in enumerate(training_set):
                def closure():
                    optimizer.zero_grad()
                    output_pred_batch = torch.cat([fno_f(input_batch),fno_s(input_batch)],dim=2).squeeze(2)
                    loss = l(output_pred_batch, output_batch) # + regularization_p *regularization(fno, 2)
                    loss.backward()
                    history.append(loss.item())
                    return loss
  
                optimizer.step(closure=closure)
                train_mse += history[-1] #loss.item() 
                
            train_mse /= len(training_set)
        
            scheduler.step()
        
            if epoch % freq_print == 0: print("######### Epoch:", epoch, " ######### Train Loss:", train_mse)
        Final_loss = round(history[-1],4)
        print('Final Loss: ', history[-1])
        return history
  
def fit(fno_f, fno_s,epochs, optimizer, training_set):  
       # history = list()
        l = torch.nn.MSELoss()
    
        for epoch in range(epochs):
            train_mse = 0.0
            for step, (input_batch, output_batch) in enumerate(training_set):
                
                optimizer.zero_grad()
                output_pred_batch = torch.cat([fno_f(input_batch),fno_s(input_batch)],dim=2).squeeze(2)
                loss = l(output_pred_batch, output_batch) 
                loss.backward()
                optimizer.step()
                #history_f.append(loss_f.item())
                train_mse += loss.item()
                
            train_mse /= len(training_set)
            scheduler.step()
        
            if epoch % freq_print == 0: print("######### Epoch:", epoch, " ######### Train Loss:", train_mse)
        #Final_loss = round(history[-1],4)
        #print('Final Loss: ', history[-1])
        #return history

def plot_hist(hist):
    plt.figure()
    plt.grid(True, which="both", ls=":")
    plt.plot(np.arange(1, len(hist) + 1), hist, label="Train Loss")
    plt.xscale("log")
    plt.legend()

In [ ]:
df = pd.read_csv('TrainingData.txt')
train_data = torch.tensor(df[['t','tf0', 'ts0']].values, dtype=torch.float)
train_data, mins, maxs = normalize(train_data)

window_len = 35
batch_size = 1
dataset = DatasetTask3(train_data, window_len)
training_set= torch.utils.data.DataLoader(dataset,batch_size=batch_size, shuffle=False)

learning_rate = 0.001

epochs = 1500
freq_print = 50
modes = 18  # we do not truncate the fourier series, rDFT of vec 35 has length 18
width = 256 

fno_f = FNO1d(modes, width)
fno_s = FNO1d(modes, width)

step_size = 50
gamma = 0.5

optimizer_Adam = Adam(list(fno_f.parameters()) + list(fno_s.parameters()) , lr=learning_rate, weight_decay= 0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer_Adam, step_size=step_size, gamma=gamma)

fit(fno_f, fno_s, epochs,optimizer_Adam, training_set) # LBFGS does not support complex valued computations
#plot_hist(hist)


######### Epoch: 0  ######### Train Loss: 0.30213421434164045
######### Epoch: 50  ######### Train Loss: 4.129579946265949e-06
######### Epoch: 100  ######### Train Loss: 1.6255502629292095e-06
######### Epoch: 150  ######### Train Loss: 1.0331160581245057e-06
######### Epoch: 200  ######### Train Loss: 8.132235592483994e-07
######### Epoch: 250  ######### Train Loss: 7.132041346835649e-07
######### Epoch: 300  ######### Train Loss: 6.628114604723123e-07
######### Epoch: 350  ######### Train Loss: 6.3600259636587e-07
######### Epoch: 400  ######### Train Loss: 6.213870818783107e-07
######### Epoch: 450  ######### Train Loss: 6.133409570452386e-07
######### Epoch: 500  ######### Train Loss: 6.088402273007887e-07
######### Epoch: 550  ######### Train Loss: 6.063799304456552e-07
######### Epoch: 600  ######### Train Loss: 6.05030818690011e-07
######### Epoch: 650  ######### Train Loss: 6.043046923309702e-07
######### Epoch: 700  ######### Train Loss: 6.039092525611523e-07
######### Epoch:

In [ ]:
test_set = train_data[-window_len:,:].reshape(1,window_len,3)
output = torch.cat([fno_f(test_set), fno_s(test_set)],dim=2).squeeze()

input_meas = denormalize(train_data, mins, maxs)
output[:,0] = (maxs[1]-mins[1])* output[:,0] + mins[1]
output[:,1] = (maxs[2]-mins[2])* output[:,1] + mins[2]

# output[:,0] = (maxs[1] - mins[1])*output[:,0] + mins[1]
# output[:,1] = (maxs[2] - mins[1])*output[:,1] + mins[1]


df = pd.read_csv('TestingData.txt')
test_time_np = df['t'].to_numpy()
test_time = torch.tensor(df['t'].values, dtype=torch.float)

total_time = torch.cat([input_meas[:,0], test_time], 0)
total_pred = torch.cat([input_meas[:,1:], output[:-1]], 0)

print(total_pred[-window_len:,1])

plt.figure()
plt.plot(input_meas[2:,0].detach(), input_meas[2:,1].detach(),label='meas Tf')
plt.plot(input_meas[2:,0].detach(), input_meas[2:,2].detach(),label='meas Ts')
plt.plot(total_time[-window_len:].detach(), total_pred[-window_len:,0].detach(), label = 'pred Tf',color='green')
plt.plot(total_time[-window_len:].detach(), total_pred[-window_len:,1].detach(), label = 'pred Ts',color='red')
plt.grid('True')
plt.legend()
plt.savefig(f'Data/task3_2fno_epochs_{epochs}_width_{width}.png')
plt.show()

In [ ]:
test_df = pd.DataFrame({'t': test_time_np, 'tf0': output[:-1,0].detach().numpy(), 'ts0': output[:-1,1].detach().numpy()})
test_df.to_csv(f'Data/task3_2fno_epochs_{epochs}_width_{width}.txt', index=False)